In [1]:
import os, sys, json, io
from pyspark.sql import *
from pyspark.sql.utils import StreamingQueryException
import sys
import json

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'
sys.path.append('/class')

# Kafka variables
brokers = 'localhost:9092'
kafka_topic = 'stocks-json'
receiver_sleep_time = 4

# Connect to Spark 
if not 'sc' in locals():
    from initspark import initspark
    sc, spark, config = initspark()

def write_memory(df, queryname = 'debug'):
    query = (df.writeStream 
            .format("memory")
            .queryName(queryname)
#            .outputMode("complete")
            .start()
            )
    return query


initializing pyspark
:: loading settings :: url = jar:file:/usr/local/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-8_2.11 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.11 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.11 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d4f74595-3f20-4de5-ab63-c18af4213cbd;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-8_2.11;2.0.2 in central
	found org.apache.kafka#kafka_2.11;0.8.2.1 in central
	found org.scala-lang.modules#scala-xml_2.11;1.0.2 in central
	found com.yammer.metrics#metrics-core;2.2.0 in central
	found org.slf4j#slf4j-api;1.7.16 in spark-list
	found org.scala-lang.modules#scala-parser-combinators_2.11;1.0.2 in central
	found com.101tec#zkclient;0.3 in central
	found log4j#log4j;1.2.17 in spark-li

pyspark initialized


In [5]:
df = (spark.readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", brokers) 
    .option("subscribe", kafka_topic) 
    .option("startingOffsets", "earliest")
    .option("failOnDataLoss", False)
    .load()
    )

# df.createOrReplaceTempView('table')
# df1 = spark.sql("""SELECT 'new data' as newfield, * from table""")
df1 = df.selectExpr("UPPER(CAST(value AS STRING)) as value")

write_memory(df1, 'debug1')

In [12]:
spark.sql("select * from debug1").take(10)

[Row(value='{"EVENT_TIME": "2022-02-18 01:18:17", "SYMBOL": "AAPL", "PRICE": 268.36, "QUANTITY": 621}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:17", "SYMBOL": "GOOG", "PRICE": 287.68, "QUANTITY": 462}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:17", "SYMBOL": "MSFT", "PRICE": 253.72, "QUANTITY": 123}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:21", "SYMBOL": "MSFT", "PRICE": 164.02, "QUANTITY": 296}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:21", "SYMBOL": "AAPL", "PRICE": 233.43, "QUANTITY": 732}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:21", "SYMBOL": "GOOG", "PRICE": 175.03, "QUANTITY": 475}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:25", "SYMBOL": "GOOG", "PRICE": 137.65, "QUANTITY": 650}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:25", "SYMBOL": "AAPL", "PRICE": 256.29, "QUANTITY": 641}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:25", "SYMBOL": "MSFT", "PRICE": 177.63, "QUANTITY": 390}'),
 Row(value='{"EVENT_TIME": "2022-02-18 01:18:29", "SYMB

In [ ]:
import os, sys, json, io
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'
sys.path.append('/class')

#from pyspark.streaming import StreamingContext
#from pyspark.streaming.kafka import KafkaUtils
import fastavro
import avro.io
import avro.schema
import avro.datafile
#import spark.sql.avro

from pyspark.sql.avro.functions import from_avro, to_avro
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Kafka variables
brokers = 'localhost:9092'
kafka_topic = 'avro-stocks'
receiver_sleep_time = 4
stock_schema = open("stock.avsc", "r").read()

from initspark import initspark
sc, spark, config = initspark()



In [ ]:
df: DataFrame = (spark.readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", brokers) 
    .option("subscribe", kafka_topic) 
    .option("startingOffsets", "earliest")
    .load()
    )

# extract the binary value of the message and convert it to the schema read from the avsc file
df1 = df.withColumn('value', from_avro("value", stock_schema))
# flatten out the value struct and remove it
df2 = df1.select(*df.columns, col("value.*")).drop("value")

# pick the columns we want to write to sql
df3 = df2.selectExpr("key as kafka_key", "timestamp as kafka_timestamp", "event_time", "symbol", "price", "quantity")



#df4 = df3.select("symbol","quantity").groupBy(window("symbol", "10 seconds")).sum("quantity")

df4 = (df3.select("kafka_timestamp", "symbol","quantity")
        .withWatermark("kafka_timestamp", "10 seconds") 
        .groupBy(window("kafka_timestamp", "10 seconds"), "symbol")
        .agg(sum("quantity").alias("sum"))
        )
print(df4)


df4 = (df3.select("kafka_timestamp", "symbol","quantity")
        .withWatermark("kafka_timestamp", "10 seconds") 
        .groupBy(window("kafka_timestamp", "10 seconds")) #.alias("group"))
        .agg(sum("quantity").alias("sum"))
        )
print(df4)

def write_memory(df, queryname = 'debug'):
    query = (df.writeStream 
            .format("memory")
            .queryName(queryname)
            .outputMode("complete")
            .start()
            )
    return query


In [ ]:
write_memory(df4)


In [ ]:
df4 = (df3.select("kafka_timestamp", "symbol","quantity")
        .withWatermark("kafka_timestamp", "10 seconds") 
        .groupBy("symbol")
        .agg(sum("quantity").alias("sum"))
        )
print(df4)

def write_memory(df, queryname = 'debug'):
    query = (df.writeStream 
            .format("memory")
            .queryName(queryname)
            .outputMode("complete")
            .start()
            )
    return query

write_memory(df4, 'debug4')

In [ ]:
x = spark.sql("select * from debug4")
x.show()


In [ ]:
spark.catalog.dropTempView("debug")

In [ ]:
spark.catalog.listTables()